In [1]:
import pandas as pd

In [6]:
df_recipes = pd.read_csv('../Datasets/RAW_recipes.csv')
df_interactions = pd.read_csv('../Datasets/RAW_interactions.csv')

In [7]:
df_grouped = df_interactions.groupby('recipe_id', group_keys=True)
df_ratings = df_grouped.agg({
    'recipe_id':'last',
    'rating':'mean'
})
df_ratings = df_ratings.reset_index(drop=True)
df_ratings = df_ratings.rename(columns={"recipe_id":"id", "rating":"score"})
df_merged = pd.merge(df_recipes, df_ratings, on='id')
df_merged = df_merged.drop(df_merged.index[[3381]]) #Error in line formatting

In [8]:
import re

set_mapping_table = str.maketrans({'[': '{', ']': '}'})
description_mapping_table = str.maketrans({"'": ""})
apostrophe_mapping_table = str.maketrans({"'": "''"})

insert_recipes_by_score = []
insert_recipes_by_number_of_steps = []
insert_recipes = []
insert_recipes_by_tag = []

string_insert_recipes_by_score = "INSERT INTO recipe.recipes_by_score (score,name,id,minutes,date_submitted) VALUES "
string_insert_recipes_by_number_of_steps = "INSERT INTO recipe.recipes_by_number_of_steps (number_of_steps,name,score,id,minutes,date_submitted) VALUES "
string_insert_recipes = "INSERT INTO recipe.recipes (id,name,minutes,contributor_id,score,date_submitted,tags,nutrition,steps,number_of_steps,description,ingredients,number_of_ingredients) VALUES "
string_insert_recipes_by_tag = "INSERT INTO recipe.recipes_by_tag (tags,name,score,id,minutes,date_submitted) VALUES "

def parseSetAndList(text, parseType):
    if parseType == 'set':
        beginning = '{'
        end = '}'
        text = text.translate(set_mapping_table)
    else:
        beginning = '['
        end = ']'
    text = text.strip(beginning + end)
    result = beginning
    for kv in re.split('\', |\", |\*|\n', text):
        kv = kv.strip()
        if(len(kv) == 0):
            continue
        elif(kv[0] == '\"'):
            kv = kv.strip("\"")
        elif(kv[0] == '\''):
            kv = kv.strip("\'")
        kv = kv.translate(apostrophe_mapping_table)
        kv = '\'' + kv +'\''
        result = result + kv + ',';
    result = result[:-1] + end
    return result
 
for index, row in df_merged.iterrows():
        insert_recipes_by_score.append(string_insert_recipes_by_score + "(" + str(row['score']) + ",'" + str(row['name']) + "'," + str(row['id']) + "," + str(row['minutes']) + ",'" + str(row['submitted']) + "');")
        insert_recipes_by_number_of_steps.append(string_insert_recipes_by_number_of_steps + "(" + str(row['n_steps']) + ",'" + str(row['name']) + "'," + str(row['score']) + "," + str(row['id']) + "," + str(row['minutes']) + ",'" + str(row['submitted']) + "');")
        insert_recipes.append((string_insert_recipes + "(" + str(row['id']) + ",'" + str(row['name']) + "'," + str(row['minutes']) + "," + str(row['contributor_id']) + "," + str(row['score']) + ",'" + str(row['submitted']) + "'," + str(row['tags']).translate(set_mapping_table) + "," + str(row['nutrition']) + "," + parseSetAndList(str(row['steps']), 'list') + "," + str(row['n_steps']) + ",'" + str(row['description']).translate(description_mapping_table) + "'," + parseSetAndList(str(row['ingredients']), 'set') + "," + str(row['n_ingredients']) + ");").replace("\n", ""))
        insert_recipes_by_tag.append(string_insert_recipes_by_tag + "(" + str(row['tags']).translate(set_mapping_table) + ",'" + str(row['name']) + "'," + str(row['score']) + "," + str(row['id']) + "," + str(row['minutes']) + ",'" + str(row['submitted']) + "');")

In [9]:
import time
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
starting_time = time.time()
session = cluster.connect()
 
try:
    for insert in insert_recipes_by_score:
        session.execute_async(insert)
    for insert in insert_recipes_by_number_of_steps:
        session.execute_async(insert)
    for insert in insert_recipes:
        session.execute_async(insert)
    for insert in insert_recipes_by_tag:
        session.execute_async(insert)
except Exception as e:
    print("An error occured : " + str(e))
    pass
elapsed_time = time.time() - starting_time
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    
cluster.shutdown()

Execution time: 00:02:32
